# Dataset selection
Loading MNIST dataset for classification digits.

In [ ]:
# loading MNIST dataset from openml
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

mnist = fetch_openml("mnist_784", as_frame=False, parser)

C:\Users\mohit\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
